In [1]:
%run secant_method.ipynb

In [2]:
def ch(x): # hyperbolic cosine
    return 0.5*(math.exp(x)+math.exp(-x))

def f(x): # the function to be approximated
    if function_type == 0: return math.exp(x)
    if function_type == 1: return math.sin(x)
    if function_type == 2: return math.cos(x)
    if function_type == 3: return math.tan(x)
    if function_type == 4: return math.cos(x) / math.sin(x)
    if function_type == 5: return math.log(x,math.e)
    if function_type == 6: return 0.2*math.exp(x) + 4*x + 1
    if function_type == 7: return 1/(1+x**3)
    if function_type == 8: return 0.5*math.tan(x) + 4*x**2 + 0.5
    return math.cos(x)/math.sin(x) + 0.6*x**2

def polinom(a,x):
    p = 0
    for j in range(m+1): p += a[j] * (x**j)
    return p

def Ch_m(v,A,a, x): # sum of polynomial and hyperbolic cosine
    sum_ai = A * ch(v*x)
    for i in range(len(a)): sum_ai += a[i] * (x**i)
    return sum_ai

def apr_expr(a,x,v='',A=0): # expression by which we approximate a given function
    if v=='': return polinom(a,x)
    return Ch_m(v,A,a, x)

In [3]:
def d_round(n,signif_figures): # round for n digits
    if abs(n) == 0.0: return 0   # -0.0
    if abs(n) > 1:
        res = round(n,signif_figures)
        if int(res) == res: return(int(res))
        return res

    str_n = str(abs(n))
    str_n = str_n[0:str_n.index('.')] + str_n[str_n.index('.')+1:]
    index = str_n.index( [value for value in list(str_n) if value != '0'][0])

    res = round(n, index+signif_figures-1)
    if int(res) == res: return int(res)

    return res

In [4]:
# to calculate the parameters of the expression

def f_f(order, n1, n2):
    if order == 0: return f(z[n1]) + f(z[n2])
    if order == 1: return f(z[n2]) - f(z[n1])
    return f_f(order-1, n1+1,n2) / M(order-1, order-1, n1+1,n2) - f_f(order-1, n1,n2-1) / M(order-1, order-1, n1,n2-1)
def E(order,v, n1, n2):
    if order == 0: return ch(v*z[n1]) + ch(v*z[n2])
    if order == 1: return ch(v*z[n2]) - ch(v*z[n1])
    return E(order-1,v, n1+1,n2) / M(order-1, order-1, n1+1,n2) - E(order-1,v, n1,n2-1) / M(order-1, order-1, n1,n2-1)
def M(l, k, n1, n2):
    if l == 1: return z[n2]**k - z[n1]**k
    return M(l-1,k, n1+1,n1+k+1) / M(l-1,l-1, n1+1,n1+k+1) - M(l-1,k, n1,n1+k) / M(l-1,1, n1,n1+k)
def w(v):
    return E(m-1,v, 1,m+1)/E(m-1,v, 0,m) - f_f(m-1, 1,m+1)/f_f(m-1, 0,m)
def A_f(v):
    return f_f(m-1, 0,m) / E(m-1,v, 0,m)
def a_2(v,A,a):
    sum_ai = 0
    for i in range(3, m+1): sum_ai += a[i-2] * (z[0]**(i-2) + z[1]**(i-2))        
    return 0.5 * ( f(z[0]) + f(z[1]) - sum_ai - A*(ch(z[0]*v) + ch(z[1]*v)) )
def a_k2(v,A,a,k):
    sum_ai = 0
    for i in range(k+3, m+1): sum_ai += a[i-2] * M(k,i-2, 0,k+1)
    return ( f_f(k, 0,k+1) - sum_ai - A * E(k,v, 0,k+1) ) / M(k,k, 0,k+1)

In [5]:
# 2 search for approximate values of the minimax approximation parameters and the error value mu at a given step
def parameters(eps,iter_vis):   # z = [z0, z1, z2]
    if eps=='':
        coeff = []
        for i in range(m+2):
            coeff.append([1])
            for j in range(m):
                coeff[i].append(z[i]**(j+1))
            coeff[i].append(((-1)**i))
        f_x = [f(z[i]) for i in range(len(z))] # f_x = y[::]
        if iter_vis == 0: print('\nparameter values at the current approximation stage:')
        res = np.linalg.solve(coeff,f_x)
        a,mu = res[:-1], res[-1]
        if iter_vis == 0:
            print('\ta:',*a,'\n\napproximate expression at the current approximation stage:',end='\n\t')
            expr_output(a)
            n_to_round = 3; mu_round = abs(d_round(mu,n_to_round))
            while mu_round == 0:
                n_to_round += 2
                mu_round = abs(d_round(mu,n_to_round))
            print('\n\nabsolute value of the approximation error at the alternation points:  |mu| =',mu_round)
        
        return '',0, a,mu

    v = secant_method(v_0,v_1,eps, iter_vis)
    A = A_f(v)
    a = np.zeros(m-1)
    for i in range(m-2,0,-1): a[i] = a_k2(v,A,a,i)
    if len(a) != 0: a[0] = a_2(v,A,a)
    mu = 0                  # error in alternance points
    for i in range(len(z)): mu += (1/len(z))*abs(f(z[i])-apr_expr(a,z[i],v,A))
    if iter_vis == 0:
        print('\tA =',A)
        for i in range(len(a)): print('\ta'+str(i+2),'=',a[i])
        print('\napproximate expression at the current approximation stage:',end='\n\t')
        expr_output(a, v,A)
        n_to_round = 3; mu_round = abs(d_round(mu,n_to_round))
        while mu_round == 0:
            n_to_round += 2
            mu_round = abs(d_round(mu,n_to_round))
        print('\n\nabsolute value of the approximation error at the alternation points:  |mu| =',mu_round)
    return v,A,a, mu

In [6]:
def ro_j(expression_type): # calculation of maximum error
    ro = 0
    for i in range(number_of_points+1):
        x_i = x_0 + i*(x_n-x_0)/number_of_points ;    f_x_i = f(x_i)
        if expression_type == 1: ro_i = abs((f_x_i - apr_expr(a,x_i,v,A)))
        else: ro_i = abs((f_x_i - apr_expr(a,x_i)))
        if ro_i > ro:
            ro, x_ro = ro_i, x_i
    return ro,x_ro

In [7]:
def visualization_functions(): # approximating and approximated function
    print('graph of the given and approximate functions:')
    fig = plt.figure(figsize=(5, 4)) # to make the description be outside of graph need to create plt.figure 
    
    yy_n = []
    for i in range(number_of_points): yy_n.append(apr_expr(a,xx[i],v,A))
    if v == '':
        plt.plot(xx,yy_n,label='approximation by a '+str(m)+'th degree expression')
    else:
        plt.plot(xx,yy_n,label='approximation by an expression with '+str(m+1)+' parameters')
    
    yy = [f(xx[i]) for i in range(number_of_points)]
    plt.plot(xx,yy,color='grey',alpha=0.6,label='given function')
    
    fig.legend(fontsize=10, bbox_to_anchor=(1., 0.8), loc='upper left', borderaxespad=-2)
    plt.xlabel('x'); plt.ylabel('y') ; plt.show()

In [8]:
 def visualization_error():
    print('error function graph:')
    fig = plt.figure(figsize=(5, 4))
    plt.plot([x_0,x_n],[0,0], lw=0.5, color='k')

    n_to_round = 2; str_mu = str(abs(d_round(mu,n_to_round)))
    while str_mu == '0':
        n_to_round += 2; str_mu = str(abs(d_round(mu,n_to_round)))
     
    plt.plot([x_0,x_n],[mu,mu],color='grey',alpha=0.3,linestyle='--',label=r'$\pm$'+r'$\|\mu\|$ '+ r'$\approx$ ' +r'$\pm$'+str_mu) # 'loosely dashdotted' is not a valid value for ls; supported values are '-', '--', '-.', ':', 'None', ' ', '', 'solid', 'dashed', 'dashdot', 'dotted'
    plt.plot([x_0,x_n],[-mu,-mu],color='grey',alpha=0.4,linestyle='--')
    
    yy_n = [apr_expr(a,xx[i],v,A)-f(xx[i]) for i in range(number_of_points)]
    plt.plot(xx,yy_n,label='error at the current stage')

    yy_alt_error = [apr_expr(a,z[i],v,A)-f(z[i]) for i in range(len(z))]
    plt.plot( z, yy_alt_error,'ok',markersize=3,label='error in current approximations\nto alternation points')
    
    fig.legend(fontsize=10, bbox_to_anchor=(1., 0.8), loc='upper left', borderaxespad=-2) # plt.legend(loc='upper left')
    plt.xlabel('x') ; plt.ylabel('y') ; plt.show()

In [9]:
def replacing_alternance_points(z, iter_vis):
    if iter_vis == 0:
        plt.figure().set_figheight(0.75)
        plt.plot(z,[0]*len(z),'ok',alpha=0.3)
    
    if x_ro < z[0]:
        # if iter_vis == 0:print('before, ', end='')
        if (apr_expr(a,x_ro,v,A)-f(x_ro)) * (apr_expr(a,z[0],v,A)-f(z[0])) >= 0:
            to_remove = z[0]
            z[0] = x_ro
            # if iter_vis == 0:print('one sign')
        else:
            to_remove = z[-1]
            z = z[0:-1]
            z.insert(0,x_ro)
            # if iter_vis == 0:print('other sign')
    elif x_ro > z[-1]:
        # if iter_vis == 0:print('after')
        if (apr_expr(a,x_ro,v,A)-f(x_ro)) * (apr_expr(a,z[-1],v,A)-f(z[-1])) >= 0:
            to_remove = z[-1]
            z[-1] = x_ro
            # if iter_vis == 0:print('one sign')
        else:
            to_remove = z[0]
            z = z[1:]
            z.append(x_ro)
            # if iter_vis == 0:print('other sign')
    
    else:
        for i in range(len(z)):
            if x_ro > z[i] and x_ro < z[i+1]: break
                
        if  (apr_expr(a,x_ro,v,A)-f(x_ro)) * (apr_expr(a,z[i],v,A)-f(z[i])) >= 0:
            to_remove= z[i]
            z.insert(z.index(z[i]), x_ro)
        else:
            to_remove = z[i+1]
            z.insert(z.index(z[i+1]), x_ro) 
        z.remove(to_remove)

    if iter_vis == 0:
        plt.plot([to_remove],[0],'xr',markersize=11)   
        plt.xlabel('x') ;  ax = plt.gca() ; ax.get_yaxis().set_visible(False) # x y axes

        print('\nchange of alternation points') ; 
        plt.plot(x_ro,0,'ok')
        plt.show()

        print('alternation points:')
        for i in range(len(z)): print(d_round(z[i],4),end=' ')
        print('\nabsolute values of errors at alternation points:')
    
        n_to_round = 2
        while (d_round(f(z[0])-apr_expr(a,z[0],v,A),n_to_round) == 0 or d_round(f(z[0])-apr_expr(a,z[1],v,A),n_to_round) == 0) and n_to_round<20:
            n_to_round += 2
        
        for i in range(len(z)): print(abs(d_round(f(z[i])-apr_expr(a,z[i],v,A),n_to_round)),end=' ')
        print('\n')
    return z

In [10]:
def final_output(a,v='',A=0):

    print('\nAs a result of the approximation, the expression obtained is:  ',end='')
    expr_output(a,v,A)

    print('\nnumber of iterations:',j+1,'\n\ncoordinates of the alternation points:  ', end='') # 
    for i in range(len(z)): print(d_round(z[i],2),end=' ')
    print('\nabsolute values of errors at alternation points:',end='\n\t')

    n_to_round = 3

    z_min = min(z)
    while d_round(f(z_min)-apr_expr(a,z_min,v,A),n_to_round) == 0 and n_to_round<20: n_to_round += 3
    for i in range(len(z)): print(abs(d_round(f(z[i])-apr_expr(a,z[i],v,A),n_to_round)),end=' ')


In [11]:
def expr_output(a, v='',A=0): # expression output
    if v=='':
        print('P =',d_round(a[0],4),end='')
        for i in range(1,len(a)):
            x_d = 'x'
            if i>1: x_d = 'x^'+str(i)
            if a[i]>=0: print(' +',str(d_round(a[i],4))+x_d,end='')
            else: print(' -',str(abs(d_round(a[i],4)))+x_d,end='')
    else:
        n_to_round = 4
        str_v = str(abs(d_round(v,n_to_round)))
        if str_v == '1': str_v = ''
        if v<0: str_v = ('(-'+str_v+')')
        print('Ch =',str(d_round(A,n_to_round))+'ch('+str_v+'x)', end='')
        for i in range(len(a)):
            if a[i] < 0: str_a = ' - '
            else: str_a = ' + '
            if abs(d_round(a[i],n_to_round)) != 1: str_a = str_a + str(abs(d_round(a[i],n_to_round)))
            if i == 1: str_a = str_a + 'x'
            elif i > 1: str_a = str_a + 'x^' + str(i)
            print(str_a, end='')